In [15]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import os
import sys
import time
import random
import math
import pandas as pd 
import seaborn as sns
import scipy.stats as stats
import scipy.optimize as optimize
import scipy.integrate as integrate
import scipy.special as special

In [16]:
import os 
import sys
import time 

In [17]:
from matplotlib.ticker import MaxNLocator, FormatStrFormatter, PercentFormatter
from datetime import datetime, date, timedelta
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import LinearRegression, HuberRegressor, Ridge, Lasso

In [18]:
train_data = pd.read_csv("C:\\Users\\user\\kaggle_playground\\data\\train.csv")

In [19]:
test_data = pd.read_csv("C:\\Users\\user\\kaggle_playground\\data\\test.csv")

In [20]:
sample_submission = pd.read_csv("C:\\Users\\user\\kaggle_playground\\data\\sample_submission.csv")

In [21]:
print(" Train data shape: ", train_data.shape)
print(" Test data shape: ", test_data.shape)
print(" Sample submission data shape: ", sample_submission.shape)
print("train_data\n", train_data.head())
print("test_data\n", test_data.head())
print("sample_submission\n", sample_submission.head())

 Train data shape:  (26298, 6)
 Test data shape:  (6570, 5)
 Sample submission data shape:  (6570, 2)
train_data
    row_id        date  country       store         product  num_sold
0       0  2015-01-01  Finland  KaggleMart      Kaggle Mug       329
1       1  2015-01-01  Finland  KaggleMart      Kaggle Hat       520
2       2  2015-01-01  Finland  KaggleMart  Kaggle Sticker       146
3       3  2015-01-01  Finland  KaggleRama      Kaggle Mug       572
4       4  2015-01-01  Finland  KaggleRama      Kaggle Hat       911
test_data
    row_id        date  country       store         product
0   26298  2019-01-01  Finland  KaggleMart      Kaggle Mug
1   26299  2019-01-01  Finland  KaggleMart      Kaggle Hat
2   26300  2019-01-01  Finland  KaggleMart  Kaggle Sticker
3   26301  2019-01-01  Finland  KaggleRama      Kaggle Mug
4   26302  2019-01-01  Finland  KaggleRama      Kaggle Hat
sample_submission
    row_id  num_sold
0   26298       100
1   26299       100
2   26300       100
3   2630

In [22]:
# null values
print("train_data null values\n", train_data.isnull().sum())

train_data null values
 row_id      0
date        0
country     0
store       0
product     0
num_sold    0
dtype: int64


In [23]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26298 entries, 0 to 26297
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   row_id    26298 non-null  int64 
 1   date      26298 non-null  object
 2   country   26298 non-null  object
 3   store     26298 non-null  object
 4   product   26298 non-null  object
 5   num_sold  26298 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 1.2+ MB


In [24]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6570 entries, 0 to 6569
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   row_id   6570 non-null   int64 
 1   date     6570 non-null   object
 2   country  6570 non-null   object
 3   store    6570 non-null   object
 4   product  6570 non-null   object
dtypes: int64(1), object(4)
memory usage: 256.8+ KB


In [11]:
from sklearn.ensemble import GradientBoostingRegressor

In [25]:
def split_month(date):
      return date.split('-')[1]

#Day
def split_day(date):
  return date.split('-')[2]

#Weekend
def weekend(date):
  import datetime
  weekend = []
  a = pd.to_datetime(date)
  for i in range(len(a)):
    if a.iloc[i].weekday() >= 5 :
      weekend.append(1)
    else:
      weekend.append(0)
  return weekend

#Weekday
def weekday(date):
    import datetime
    weekday = []
    a = pd.to_datetime(date)
    for i in range(len(a)):
        weekday.append(a.iloc[i].weekday())
    return weekday
train_data['Month'] = train_data['date'].apply(split_month)
train_data['Day'] = train_data['date'].apply(split_day)
train_data['Weekend'] = weekend(train_data['date'])
train_data['Weekday'] = weekday(train_data['date'])
train_data = train_data.drop(columns = ['row_id', 'date'])

test_data['Month'] = test_data['date'].apply(split_month)
test_data['Day'] = test_data['date'].apply(split_day)
test_data['Weekend'] = weekend(test_data['date'])
test_data['Weekday'] = weekday(test_data['date'])
test_data = test_data.drop(columns = ['row_id', 'date'])
#Dummies the 'country', 'store', 'product'
train_data_dum = pd.get_dummies(train_data[['country', 'store', 'product']])
test_data_dum = pd.get_dummies(test_data[['country', 'store', 'product']])

train_data = pd.concat([train_data, train_data_dum],axis = 1)
test_data = pd.concat([test_data, test_data_dum],axis = 1)

train_data = train_data.drop(columns = ['country', 'store', 'product'])
test_data = test_data.drop(columns = ['country', 'store', 'product'])

data = train_data.drop(columns = 'num_sold')
target = train_data['num_sold']

from sklearn.preprocessing import StandardScaler
Normalize = StandardScaler()
target = np.log(target)
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data, target, train_size = 0.8, random_state = 5)
x_train = Normalize.fit_transform(x_train)
x_test = Normalize.transform(x_test)

In [26]:
GBR = GradientBoostingRegressor(learning_rate=0.10, max_depth= 6,
                                min_samples_leaf = 5,n_estimators = 500, random_state = 40,subsample = 0.3).fit(x_train, y_train)

In [27]:
y_pred_GBR = GBR.predict(x_test)

In [28]:
Model = GBR.fit(x_train, y_train)


In [29]:
test_data = Normalize.transform(test_data)

In [30]:
submission_target= np.exp(GBR.predict(test_data))

In [31]:
sample_submission['num_sold'] = submission_target


In [27]:
sample_submission.to_csv('submission.csv',index=False)